In [14]:
import matplotlib.pyplot as plt
import numpy as np
import copy
import math
import csv
from google.colab import files
from collections import deque
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import cross_val_score

In [2]:
uploaded = files.upload()

Saving hERG_Sample_Discrete_Dataset.csv to hERG_Sample_Discrete_Dataset.csv


In [62]:
file_data   = "hERG_Sample_Discrete_Dataset.csv"
handle_file = open(file_data, "r")
data        = handle_file.readlines()
handle_file.close()

size_row    = 2245    # 각 데이터의 요소
size_col    = 6095    # 데이터 갯수

num_image   = len(data)   #10000개
count       = 0     # count for the number of images

#
# normalize the values of the input data to be [0, 1]
#
def normalize(data):

    data_normalized = (data - min(data)) / (max(data) - min(data))

    return(data_normalized)


#
# make a matrix each column of which represents an images in a vector form
#
X = np.empty((6095, 2245))
label = np.empty(6095)

for line in data:
  if count == 0:
    count += 1
    continue
  else:  
    line_data   = line.split(',')
    #print(len(line_data))
    data_label  = line_data[2246]
    #print(data_label)
    data_vector = np.asfarray(line_data[1:2246])
    #print(len(data_vector))
    #print(data_vector)
    data_vector = normalize(data_vector)

    label[count-1] = data_label       # size : 10000
    X[count-1, :]    = data_vector   # size : 6095 * 2245

    count += 1

X = torch.from_numpy(X)
X = X.type(torch.float32)
label = torch.from_numpy(label)
label = label.type(torch.float32)         #float32 지정을 통해 backward 오류 해결!

train_costs = deque()
test_costs = deque()
iters = deque()
train_accuracys = deque()
test_accuracys = deque()

In [63]:
print(X[6094])
print(label[6094])

tensor([0.0267, 0.0009, 0.0027,  ..., 0.0040, 0.0348, 0.0147])
tensor(0.)


In [15]:
class NeuralNet(torch.nn.Module):
  def __init__(self, D_in, H, D_out):
    super(NeuralNet, self).__init__()
    self.linear1 = nn.Linear(D_in, H)
    self.linear2 = nn.Linear(H, D_out)

  def forward(self, x):
    h_relu = self.linear1(x).clamp(min=0)
    y_pred = self.linear2(h_relu)
    return y_pred

In [64]:
def main():
  D_in, H, D_out = 2245, 40, 1
  model = NeuralNet(D_in, H, D_out)

  criterion = nn.MSELoss()
  optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
  
  for iter in range(500):
      y_pred = model(X.float())  
      optimizer.zero_grad()
      loss = criterion(y_pred, label)
      #print(loss)
      loss.backward()
      optimizer.step()
if __name__ == "__main__":
  main()

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/loss.py:432: UserWarning: Using a target size (torch.Size([6095])) that is different to the input size (torch.Size([6095, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


tensor(0.5690, grad_fn=<MseLossBackward>)
tensor(0.5679, grad_fn=<MseLossBackward>)
tensor(0.5665, grad_fn=<MseLossBackward>)
tensor(0.5652, grad_fn=<MseLossBackward>)
tensor(0.5641, grad_fn=<MseLossBackward>)
tensor(0.5627, grad_fn=<MseLossBackward>)
tensor(0.5615, grad_fn=<MseLossBackward>)
tensor(0.5608, grad_fn=<MseLossBackward>)
tensor(0.5602, grad_fn=<MseLossBackward>)
tensor(0.5596, grad_fn=<MseLossBackward>)
tensor(0.5587, grad_fn=<MseLossBackward>)
tensor(0.5576, grad_fn=<MseLossBackward>)
tensor(0.5564, grad_fn=<MseLossBackward>)
tensor(0.5551, grad_fn=<MseLossBackward>)
tensor(0.5536, grad_fn=<MseLossBackward>)
tensor(0.5526, grad_fn=<MseLossBackward>)
tensor(0.5514, grad_fn=<MseLossBackward>)
tensor(0.5507, grad_fn=<MseLossBackward>)
tensor(0.5502, grad_fn=<MseLossBackward>)
tensor(0.5497, grad_fn=<MseLossBackward>)
tensor(0.5491, grad_fn=<MseLossBackward>)
tensor(0.5486, grad_fn=<MseLossBackward>)
tensor(0.5481, grad_fn=<MseLossBackward>)
tensor(0.5477, grad_fn=<MseLossBac